In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [5]:
train = pd.read_pickle(FILE.train_base_line.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_base_line.value)
print('test shape is: {}'.format(test.shape))

X = pd.concat([train,test],sort=False)
print(X.shape)

X = X.sort_values('time')
X = X.reset_index(drop=True)


print(X.shape)

train shape is: (1001650, 33)
test shape is: (40024, 32)
(1041674, 33)
(1041674, 33)


# Define Function List

In [3]:
count_func_list = [count,cumulative_count,reverse_cumulative_count]
time_next_func_list = [time_to_n_next,time_to_n_previous]
count_time_func_list = [count_in_previous_n_time_unit,count_in_next_n_time_unit]

# FillNa

In [24]:
le = LabelEncoder()
#X['user_tags'] =  X['user_tags'].fillna('naNotFound')
# X['user_tags'] = le.fit_transform(X['user_tags'])

for col in tqdm(categorical_cols):
    if str(X[col].dtype) == 'category':
        X[col] = X[col].astype(str)
        X[col] = X[col].fillna('FillNAValue')
        pass
    else:
        X[col] = X[col].fillna(-100000)
    X[col] = le.fit_transform(X[col])

100%|██████████| 7/7 [00:05<00:00,  1.22it/s]


In [6]:
base_line_split = pd.read_csv('../../data/features/feature_importance_split_baseline.csv')
base_line_gain = pd.read_csv('../../data/features/feature_importance_gain_baseline.csv')

In [7]:
NUM = 5
categorical_cols = list(set(list(base_line_split.feature.values[:NUM]) + list(base_line_gain.feature.values[:NUM])))

# Aggregation

In [8]:
def get_new_name(cob_col,func):
    return '+'.join(cob_col)+'_'+func.__name__

In [27]:
def auto_generating(X,categorical_cols,
                    count_func_list,time_next_func_list,count_time_func_list,
                    dummy_col='instance_id',time_col='time',n_comb=1,
                    saving_path_count=None,
                    saving_path_timeNext=None,
                    saving_path_timeCount=None):
    
    if saving_path_count is not None:
        try:
            result_count = pd.read_pickle(saving_path_count)
        except:
            result_count = pd.DataFrame()
            
    if saving_path_timeNext is not None:
        try:
            result_time_next = pd.read_pickle(saving_path_timeNext)
        except:
            result_time_next = pd.DataFrame()
            
    if saving_path_timeCount is not None:
        try:
            result_time_count = pd.read_pickle(saving_path_timeCount)
        except:
            result_time_count = pd.DataFrame()

    counter = 0
    for col in combinations(categorical_cols,n_comb):
        print('#######   {}/{}   ########'.format(col,counter))
        col = list(col)
        counter+=1
        need_save_result_count = False
        need_save_result_time_next = False
        need_save_result_time_count = False
        
        for func in count_func_list:
            feature_name = get_new_name(col,func)
            if feature_name in result_count.columns:
                continue
            result_count[feature_name] = func(X,col,dummy_col,feature_name)[feature_name]
            need_save_result_count = True

        for func in time_next_func_list:

            feature_name = get_new_name(col,func)
            col_time = col.copy()
            col_time.append(time_col)
            if feature_name in result_time_next.columns:
                continue
            result_time_next[feature_name] = func(X,col_time,dummy_col,feature_name)[time_col]
            need_save_result_time_next = True


        for func in count_time_func_list:
#             continue

            feature_name = get_new_name(col,func)
            col_time = col.copy()
            col_time.append(time_col)
            if feature_name+'_1hour' not in result_time_count.columns:
                result_time_count[feature_name+'_1hour'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*60*1})[feature_name]
                need_save_result_time_count = True
            if feature_name+'_5min' not in result_time_count.columns:
                result_time_count[feature_name+'_5min'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*5})[feature_name]
                need_save_result_time_count = True

            if feature_name+'_30min' not in result_time_count.columns:
                result_time_count[feature_name+'_30min'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*30})[feature_name]
                need_save_result_time_count = True
                
        if saving_path_count is not None and need_save_result_count:
            result_count[dummy_col]=X[dummy_col].copy()
            print('saving counts')
            result_count.to_pickle(saving_path_count)
        else:
            print('skip counts')

        if saving_path_timeNext is not None and need_save_result_time_next:
            result_time_next[dummy_col]=X[dummy_col].copy()
            print('saving time next')
            result_time_next.to_pickle(saving_path_timeNext)
        else:
            print('skip time next')

        if saving_path_timeCount is not None and need_save_result_time_count:
            result_time_count[dummy_col]=X[dummy_col].copy()
            print('saving time count in next unit')
            result_time_count.to_pickle(saving_path_timeCount)
        
        else:
            print('skip time count in next unit')
            
    

In [28]:
dummy_col = 'instance_id'
time_col = 'time'
X = X.sort_values(time_col)

for n_combs in range(1,len(categorical_cols)+1):
    saving_path_count = FILE.X_fe_agg_count_formaterV3.value.format(n_combs)
    saving_path_timeNext = FILE.X_fe_agg_time_next_formaterV3.value.format(n_combs)
    saving_path_timeCount = FILE.X_fe_agg_time_count_formaterV3.value.format(n_combs)
    print(saving_path_count)
    print(saving_path_timeNext)
    print(saving_path_timeCount)
    
    auto_generating(X,
                    categorical_cols,
                    count_func_list,
                    time_next_func_list,
                    count_time_func_list,
                    dummy_col,
                    time_col,
                    n_comb=n_combs,
                    saving_path_count=saving_path_count,
                    saving_path_timeNext=saving_path_timeNext,
                    saving_path_timeCount=saving_path_timeCount)

../LIB/../../data/features/X_fe_aggV3_1_count.pkl
../LIB/../../data/features/X_fe_aggV3_1_time_next.pkl
../LIB/../../data/features/X_fe_aggV3_1_time_count.pkl
#######   ('model',)/0   ########
skip counts
skip time next
saving time count in next unit
#######   ('app_id',)/1   ########
skip counts
skip time next
saving time count in next unit
#######   ('city',)/2   ########
skip counts
skip time next
saving time count in next unit
#######   ('advert_name',)/3   ########
skip counts
skip time next
saving time count in next unit
#######   ('user_tags',)/4   ########
skip counts
skip time next
saving time count in next unit
#######   ('creative_type',)/5   ########
skip counts
skip time next
saving time count in next unit
#######   ('osv',)/6   ########
skip counts
skip time next
saving time count in next unit
../LIB/../../data/features/X_fe_aggV3_2_count.pkl
../LIB/../../data/features/X_fe_aggV3_2_time_next.pkl
../LIB/../../data/features/X_fe_aggV3_2_time_count.pkl
#######   ('model', 'a

saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'advert_name', 'user_tags')/4   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'advert_name', 'creative_type')/5   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'advert_name', 'osv')/6   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'user_tags', 'creative_type')/7   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'user_tags', 'osv')/8   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'app_id', 'creative_type', 'osv')/9   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 'city', 'advert_name', 'user_tags')/10   ########
saving counts
saving time next
saving time count in next unit
#######   ('model', 

saving counts
saving time next
saving time count in next unit
#######   ('model', 'city', 'advert_name', 'user_tags', 'creative_type', 'osv')/5   ########
saving counts
saving time next
saving time count in next unit
#######   ('app_id', 'city', 'advert_name', 'user_tags', 'creative_type', 'osv')/6   ########
saving counts
saving time next
saving time count in next unit
../LIB/../../data/features/X_fe_aggV3_7_count.pkl
../LIB/../../data/features/X_fe_aggV3_7_time_next.pkl
../LIB/../../data/features/X_fe_aggV3_7_time_count.pkl
#######   ('model', 'app_id', 'city', 'advert_name', 'user_tags', 'creative_type', 'osv')/0   ########
saving counts
saving time next
saving time count in next unit
